In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv("raw_data/2026-01-13/norm_indicators_BTCUSDT.csv")
data

,Timestamp,RSI_14,MACD_HIST,BB_POSITION,RET_1,RET_5,RET_15,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
0,2021-02-03 08:00:00,0.241043,0.766984,0.743267,-0.377756,0.564120,1.085109,0.886003,-1.229179,0.436301,-0.324545,-0.385126,-0.304051,0.002224,0.424719
1,2021-02-03 12:00:00,0.331490,0.873115,0.951573,0.780077,0.431135,1.214219,1.406328,-1.195125,0.383037,-0.116924,0.772595,0.814413,0.062215,0.146598
2,2021-02-03 16:00:00,0.319736,0.813402,0.809429,-0.156423,0.548618,1.047009,1.168740,-1.349722,0.313561,-0.121001,0.440396,-0.058460,0.355312,0.139496
3,2021-02-03 20:00:00,0.397526,0.948178,0.958863,0.713091,0.456294,1.191007,1.598814,-1.327871,0.260244,-0.461940,1.474987,0.795460,0.041940,0.138442
4,2021-02-04 00:00:00,0.373060,0.891754,0.817041,-0.184935,0.446622,1.567032,1.296278,-1.335252,0.108684,-0.046030,-0.518228,-0.117380,0.645205,0.131033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10825,2026-01-12 12:00:00,0.120618,0.628817,1.060308,1.827616,0.750075,0.185831,0.174942,-0.243553,-0.752349,0.531128,0.743218,1.206470,0.000000,0.569129
10826,2026-01-12 16:00:00,0.153317,0.768563,1.079844,0.295892,0.633060,0.251728,0.292087,-0.019464,-0.781664,1.094078,0.987219,0.218412,0.523808,0.739383
10827,2026-01-12 20:00:00,0.034096,0.653073,0.429864,-1.056101,-0.634100,-0.022114,-0.210253,0.076271,-0.651300,-0.526775,-0.354414,-0.618494,0.202447,0.391847
10828,2026-01-13 00:00:00,0.055078,0.595243,0.505608,0.170832,-0.124544,0.068910,-0.115116,-0.132896,-0.654836,-0.595467,0.178072,0.127454,0.155113,0.286705


In [9]:
indicators = pd.read_csv("raw_data/2026-01-12/indicators_BTCUSDT.csv")
indicators

,Timestamp,RSI_14,MACD_HIST,BB_POSITION,RET_1,RET_5,RET_15,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
0,2021-01-23 00:00:00,44.024230,85.613790,0.358264,-0.017767,0.048186,-0.054919,-973.265756,1509.170765,0.025080,12211.522030,-99.400000,-0.387623,0.016493,0.206524
1,2021-01-23 04:00:00,47.167103,139.138290,0.459167,0.017342,0.037760,-0.066544,-370.087051,1486.296425,0.024780,11464.516691,263.231097,0.380933,0.353106,0.065889
2,2021-01-23 08:00:00,41.472528,92.485818,0.283500,-0.039824,-0.025364,-0.114042,-1505.251865,1493.968109,0.025198,13454.305895,-812.031030,-0.860581,0.116314,0.089861
3,2021-01-23 12:00:00,42.811997,82.109707,0.338499,0.007292,-0.052326,-0.081130,-1157.856241,1451.476815,0.025191,10854.235249,-443.489071,0.160664,0.283952,0.174815
4,2021-01-23 16:00:00,45.545328,110.149050,0.433512,0.014689,-0.018268,-0.067978,-623.787491,1400.682043,0.024760,9253.459243,47.241291,0.336757,0.107955,0.083852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10883,2026-01-10 20:00:00,43.734010,-19.627799,0.287023,-0.001345,-0.001341,0.003076,-444.611224,812.470197,0.005749,491.475500,-128.162348,-0.149962,0.039989,0.123943
10884,2026-01-11 00:00:00,46.693114,0.116686,0.489953,0.002525,0.000606,-0.000530,-196.192022,774.925897,0.005750,463.880390,92.650000,0.295254,0.005742,0.069168
10885,2026-01-11 04:00:00,46.432091,13.353828,0.498218,-0.000250,-0.000144,-0.002495,-199.001838,738.478333,0.005738,546.843420,29.617988,-0.030739,0.157770,0.169876
10886,2026-01-11 08:00:00,47.052476,26.057987,0.544740,0.000487,0.000984,-0.003794,-140.728943,700.424881,0.005715,694.376170,55.119940,0.063090,0.087104,0.143527


In [10]:
indicators.head()

,Timestamp,RSI_14,MACD_HIST,BB_POSITION,RET_1,RET_5,RET_15,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
0,2021-01-23 00:00:00,44.024230,85.613790,0.358264,-0.017767,0.048186,-0.054919,-973.265756,1509.170765,0.025080,12211.522030,-99.400000,-0.387623,0.016493,0.206524
1,2021-01-23 04:00:00,47.167103,139.138290,0.459167,0.017342,0.037760,-0.066544,-370.087051,1486.296425,0.024780,11464.516691,263.231097,0.380933,0.353106,0.065889
2,2021-01-23 08:00:00,41.472528,92.485818,0.283500,-0.039824,-0.025364,-0.114042,-1505.251865,1493.968109,0.025198,13454.305895,-812.031030,-0.860581,0.116314,0.089861
3,2021-01-23 12:00:00,42.811997,82.109707,0.338499,0.007292,-0.052326,-0.081130,-1157.856241,1451.476815,0.025191,10854.235249,-443.489071,0.160664,0.283952,0.174815
4,2021-01-23 16:00:00,45.545328,110.149050,0.433512,0.014689,-0.018268,-0.067978,-623.787491,1400.682043,0.024760,9253.459243,47.241291,0.336757,0.107955,0.083852


In [13]:
norm = pd.read_csv("raw_data/2026-01-12/norm_indicators_BTCUSDT.csv")
norm

,Timestamp,RSI_14,MACD_HIST,BB_POSITION,RET_1,RET_5,RET_15,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
0,2021-02-01 20:00:00,0.009561,-0.794848,-0.192871,-0.338028,-0.104356,-0.156795,-0.016089,-0.617622,0.647162,-0.910174,-0.340463,-0.252599,0.162752,0.077467
1,2021-02-02 00:00:00,0.012815,-0.765626,-0.134155,0.004073,-0.214784,-0.414570,-0.011356,-0.783619,0.598386,-0.654929,-0.276787,0.027711,0.509587,0.082634
2,2021-02-02 04:00:00,0.102107,-0.503067,0.595597,0.682275,-0.017873,-0.151961,0.512011,-0.872260,0.588494,-0.523714,0.490804,0.568272,0.149960,0.045917
3,2021-02-02 08:00:00,0.162332,-0.176738,0.961132,0.464163,0.905281,0.059730,0.831292,-0.686037,0.432285,0.593619,0.467388,0.411544,0.785319,0.118782
4,2021-02-02 12:00:00,0.160361,-0.000066,0.848918,-0.063327,0.446952,0.192110,0.712570,-0.794242,0.416019,-0.318351,0.298726,-0.012259,0.403307,0.317442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10824,2026-01-10 20:00:00,-0.125320,0.017155,-0.425953,-0.327000,-0.301080,-0.192580,-0.821668,-0.570354,-1.844743,-1.167332,-0.441740,-0.149962,0.039989,0.123943
10825,2026-01-11 00:00:00,-0.066138,0.108771,-0.020094,0.345351,-0.154723,-0.341419,-0.601085,-0.819724,-1.773154,-1.176903,0.088593,0.295254,0.005742,0.069168
10826,2026-01-11 04:00:00,-0.071358,0.166265,-0.003564,-0.143929,-0.242871,-0.419131,-0.607345,-1.060003,-1.732566,-1.105802,-0.070255,-0.030739,0.157770,0.169876
10827,2026-01-11 08:00:00,-0.058950,0.221592,0.089480,-0.003844,-0.147899,-0.463574,-0.552564,-1.305816,-1.711826,-1.000701,-0.001417,0.063090,0.087104,0.143527


In [12]:
normalised = pd.read_csv("raw/normalized_indicators/normalized_AAPL.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'raw/normalized_indicators/normalized_AAPL.csv'

In [ ]:
normalised.head()

,Date,SMA_20,SMA_50,EMA_12,EMA_26,RSI_14,MACD,MACD_SIGNAL,BB_UPPER,BB_LOWER,BB_MIDDLE,ADX_14,OBV
0,2023-01-03,0.0,0.0,0.0,0.0,1.000000,0.944773,0.899225,0.0,0.0,0.0,1.0,0.032561
1,2023-01-04,0.0,0.0,0.0,0.0,1.000000,0.944773,0.899225,0.0,0.0,0.0,1.0,0.032561
2,2023-01-05,0.0,0.0,0.0,0.0,0.905116,0.944773,0.899225,0.0,0.0,0.0,1.0,0.000000
3,2023-01-06,0.0,0.0,0.0,0.0,0.925496,0.944773,0.899225,0.0,0.0,0.0,1.0,0.035293
4,2023-01-09,0.0,0.0,0.0,0.0,0.927430,0.944773,0.899225,0.0,0.0,0.0,1.0,0.063764


In [ ]:
indicators = pd.read_csv("raw/indicators/indicators_AAPL.csv")
indicators

,Date,SMA_20,SMA_50,EMA_12,EMA_26,RSI_14,MACD,MACD_SIGNAL,BB_UPPER,BB_LOWER,BB_MIDDLE,ADX_14,OBV
0,2023-01-03,135.778999,144.7934,131.252500,139.408845,100.000000,8.209849,7.095565,149.050769,122.507230,135.778999,74.113995,8.911360e+07
1,2023-01-04,135.778999,144.7934,131.252500,139.408845,100.000000,8.209849,7.095565,149.050769,122.507230,135.778999,74.113995,8.911360e+07
2,2023-01-05,135.778999,144.7934,131.252500,139.408845,92.600758,8.209849,7.095565,149.050769,122.507230,135.778999,74.113995,8.150900e+06
3,2023-01-06,135.778999,144.7934,131.252500,139.408845,94.190027,8.209849,7.095565,149.050769,122.507230,135.778999,74.113995,9.590560e+07
4,2023-01-09,135.778999,144.7934,131.252500,139.408845,94.340849,8.209849,7.095565,149.050769,122.507230,135.778999,74.113995,1.666964e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,2024-10-23,229.727000,226.4908,231.806516,229.301200,52.593026,2.505316,2.163433,237.575042,221.878958,229.727000,15.860638,2.282126e+09
455,2024-10-24,229.879500,226.6678,231.616284,229.395186,52.271541,2.221098,2.174966,237.665257,222.093743,229.879500,15.128280,2.251016e+09
456,2024-10-25,230.060500,226.8016,231.584548,229.544431,53.621399,2.040117,2.147996,237.809955,222.311045,230.060500,14.983254,2.289818e+09
457,2024-10-28,230.080500,226.9486,231.863848,229.830029,56.742652,2.033819,2.125161,237.863887,222.297113,230.080500,15.165303,2.325906e+09


In [ ]:
import yfinance as yf
print(yf.__version__)

data = yf.download("AAPL", start="2010-01-01", interval="1d")
print(data.index.min(), data.index.max(), len(data))

0.2.66


/tmp/ipykernel_45826/280823695.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2010-01-01", interval="1d")
[*********************100%***********************]  1 of 1 completed

2010-01-04 00:00:00 2025-12-19 00:00:00 4017


In [ ]:
data

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.418383,6.433078,6.369497,6.400988,493729600
2010-01-05,6.429479,6.465768,6.395588,6.436077,601904800
2010-01-06,6.327210,6.454972,6.320612,6.429479,552160000
2010-01-07,6.315515,6.358102,6.269628,6.350604,477131200
2010-01-08,6.357502,6.358103,6.269929,6.307118,447610800
...,...,...,...,...,...
2025-12-15,274.109985,280.149994,272.839996,280.149994,50409100
2025-12-16,274.609985,275.500000,271.790009,272.820007,37648600
